# Processador de planilha
- Recebe uma planilha obtida de https://dados.gov.br/dados/conjuntos-dados/previsao-climatica-do-censipam
- Lê os registros
- Realiza composições
- Faz inserções no BD

In [1]:
import pandas as pd
df = pd.read_csv('../documentos/previsaoTempoDadoAbertoOut2022_gerado_01112022_100000.csv')

In [2]:
df.columns

Index(['Data', 'Cidade', 'Condicao_do_Tempo_Prevista', 'Temperatura_Tendencia',
       'Temperatura_Minima', 'Temperatura_Maxima', 'Vento_Velocidade_Minima',
       'Vento_Velocidade_Maxima', 'Vento_Direcao', 'Vento_Intensidade'],
      dtype='object')

In [3]:
df['Condicao_do_Tempo_Prevista'].unique()

array(['Claro a nublado com pancadas de chuva isoladas a tarde',
       'Claro a nublado com pancadas de chuva isoladas a noite',
       'Poucas nuvens a nublado com possibilidade de chuva em areas isoladas a noite',
       'Poucas nuvens a nublado com pancadas de chuva e trovoadas isoladas a noite',
       'Poucas nuvens a nublado com pancadas de chuva e trovoadas isoladas a tarde e a noite',
       'Parcialmente nublado a nublado com pancadas de chuva e trovoadas isoladas a tarde e a noite',
       'Poucas nuvens a nublado com pancadas de chuva e trovoadas isoladas a tarde',
       'Claro a nublado com pancadas de chuva e trovoadas a tarde',
       'Poucas nuvens a nublado com pancadas de chuva e trovoadas a tarde',
       'Poucas nuvens a nublado com pancadas de chuva e trovoadas isoladas entre o final da tarde e o inicio da noite',
       'Poucas nuvens a nublado',
       'Poucas nuvens a nublado com pancadas de chuva isoladas ao final da tarde',
       'Parcialmente nublado a nubl

In [4]:
import mysql.connector
#python -m pip install mysql-connector-python
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="amaz123",
  database="amazonia_tempo"
)
#alter user 'root'@'localhost' IDENTIFIED WITH mysql_native_password by 'amaz123';

In [5]:
df

,Data,Cidade,Condicao_do_Tempo_Prevista,Temperatura_Tendencia,Temperatura_Minima,Temperatura_Maxima,Vento_Velocidade_Minima,Vento_Velocidade_Maxima,Vento_Direcao,Vento_Intensidade
0,04/10/2022,Abaetetuba - PA,Claro a nublado com pancadas de chuva isoladas...,Estavel,24,34,50,95,SE - E,Fracos/Moderados
1,05/10/2022,Abaetetuba - PA,Claro a nublado com pancadas de chuva isoladas...,Estavel,24,34,50,95,SE - E,Fracos/Moderados
2,06/10/2022,Abaetetuba - PA,Claro a nublado com pancadas de chuva isoladas...,Estavel,24,34,50,95,SE - E,Fracos/Moderados
3,07/10/2022,Abaetetuba - PA,Poucas nuvens a nublado com possibilidade de c...,Estavel,24,34,50,95,SE - E,Fracos/Moderados
4,08/10/2022,Abaetetuba - PA,Poucas nuvens a nublado com pancadas de chuva ...,Estavel,24,34,50,95,SE - E,Fracos/Moderados
...,...,...,...,...,...,...,...,...,...,...
8841,27/10/2022,Xinguara - PA,Parcialmente nublado a nublado com chuva e tro...,Estavel,23,32,60,100,SE - NE,Fracos/Moderados
8842,28/10/2022,Xinguara - PA,Nublado a parcialmente nublado com chuva e tro...,Estavel,23,31,65,100,SE - NE,Fracos/Moderados
8843,29/10/2022,Xinguara - PA,Parcialmente nublado a nublado com pancadas de...,Estavel,23,32,60,100,SE - NE,Fracos/Moderados
8844,30/10/2022,Xinguara - PA,Parcialmente nublado a nublado com pancadas de...,Estavel,23,32,60,100,SE - NE,Fracos/Moderados


In [6]:
def tbl_data(x_data):
    #mycursor = mydb.cursor()
    sql = "INSERT INTO data (dia, mes, ano) VALUES (%s, %s, %s)"
    d, m, a = x_data.split("/")
    val = (d.strip(), m.strip(), a.strip())
    #mycursor.execute(sql, val)
    #mydb.commit()
    #return mycursor.lastrowid

In [7]:
def tbl_localizacao(x_local):
    mycursor = mydb.cursor()
    sql = "INSERT INTO localizacao (cidade, estado, pais) VALUES (%s, %s, %s)"
    cidade, uf = x_local.split('-')
    val = (cidade.strip(), uf.strip(), 'BRASIL')
    mycursor.execute(sql, val)
    mydb.commit()
    return mycursor.lastrowid

In [14]:
def tbl_temperatura(x_temp):
    mycursor = mydb.cursor()
    sql = "INSERT INTO temperatura (temperatura_tendencia) VALUES (%s)"
    val = tuple(x_temp.strip())
    print(val)
    mycursor.execute(sql, val)
    mydb.commit()
    return mycursor.lastrowid

In [15]:
def tbl_vento(x_vento_dir, x_vento_int):
    mycursor = mydb.cursor()
    sql = "INSERT INTO Vento (temperatura_tendencia) VALUES (%s, %s)"
    val = (x_vento_dir.strip(), x_vento_int.strip())
    mycursor.execute(sql, val)
    mydb.commit()
    return mycursor.lastrowid

In [16]:
def medias(local):
    df_cidade = df[df['Cidade']==local];
    t_min = df_cidade[['Temperatura_Minima']].mean()
    t_max = df_cidade[['Temperatura_Maxima']].mean()
    v_min = df_cidade[['Vento_Velocidade_Minima']].mean()
    v_max = df_cidade[['Vento_Velocidade_Maxima']].mean()
    
    return t_min, t_max, v_min, v_max

In [17]:
def inserir_linha(row):
    id_data  = tbl_data(row['Data'])
    id_local = tbl_localizacao(row['Cidade'])
    id_temp  = tbl_temperatura(row['Temperatura_Tendencia'])
    id_vento = tbl_vento(row['Vento_Direcao'], row['Vento_Intensidade'])
    
    t_min, t_max, v_min, v_max = medias(row['Cidade'])
    
    mycursor = mydb.cursor()
    sql = "INSERT INTO Tempo (id_data, id_localizacao, id_temperatura, id_vento, "
    sql +=" vento_velocidade_minima, vento_velocidade_maxima, "
    sql +=" temperatura_Minima, temperatura_Maxima, condicao_do_Tempo_Prevista, "
    sql += " Temperatura_Minima, Temperatura_Maxima, Vento_Velocidade_Minima, Vento_Velocidade_Maxima)"
    sql += " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    val = (id_data, id_local, id_temp, id_vento, 
            row['Vento_Velocidade_Minima'],row['Vento_Velocidade_Maxima'],
            row['Temperatura_Minima'], row['Temperatura_Maxima'],
            row['Condicao_do_Tempo_Prevista'],
            t_min, t_max, v_min, v_max)
    mycursor.execute(sql, val)
    mydb.commit()
    return mycursor.lastrowid


In [18]:
df.apply(lambda x: inserir_linha(x), axis = 1)

('E', 's', 't', 'a', 'v', 'e', 'l')


ProgrammingError: Not all parameters were used in the SQL statement